```{contents}
```
## Timeout

A **timeout** in LangGraph is a **hard execution limit** that automatically **terminates a node, branch, or entire graph run** if it exceeds a specified duration.
Timeouts are a **core production-safety mechanism** used to prevent runaway execution, infinite loops, resource exhaustion, and degraded system performance.

---

### **1. Why Timeouts Exist**

LLM systems interact with **external models, tools, networks, and humans**.
Any of these can stall, hang, or slow down.

| Risk            | Failure Without Timeout |
| --------------- | ----------------------- |
| Stuck tool call | Graph never completes   |
| Infinite loop   | Cost explosion          |
| Slow LLM        | Thread exhaustion       |
| Human delay     | Pipeline freeze         |
| Network failure | Resource leak           |

Timeouts convert **unbounded uncertainty** into **bounded failure**.

---

### **2. Timeout Scopes in LangGraph**

LangGraph supports timeouts at multiple layers:

| Scope                 | Purpose                  |
| --------------------- | ------------------------ |
| Node-level timeout    | Limit a single operation |
| Edge / branch timeout | Limit a sub-workflow     |
| Graph-level timeout   | Limit full execution     |
| Loop timeout          | Prevent infinite cycles  |
| External tool timeout | Guard external calls     |

---

### **3. Graph-Level Timeout**

Controls total runtime of the graph execution.

```python
result = graph.invoke(
    input,
    config={"timeout": 30}   # seconds
)
```

If exceeded → **graph aborts with TimeoutError**.

---

### **4. Node-Level Timeout**

Applied to individual nodes.

```python
def slow_node(state):
    time.sleep(10)
    return {"x": 1}

builder.add_node("slow", slow_node, timeout=5)
```

If `slow_node` runs > 5 seconds → **node fails** and control transfers to error handling.

---

### **5. Timeout with Retry & Fallback**

```python
builder.add_node(
    "llm_call",
    llm_node,
    timeout=15,
    retry=3,
    fallback="safe_response"
)
```

Execution policy:

```
Attempt → Timeout → Retry → Timeout → Retry → Fallback
```

---

### **6. Loop Timeout Protection**

Critical for cyclic graphs.

```python
graph.invoke(input, config={
    "recursion_limit": 20,
    "timeout": 60
})
```

| Protection      | Purpose              |
| --------------- | -------------------- |
| recursion_limit | Stops infinite loops |
| timeout         | Stops long loops     |

---

### **7. External Tool Timeout Example**

```python
def api_call(state):
    response = requests.get(url, timeout=3)
    return {"data": response.json()}
```

LangGraph propagates this timeout into the graph's execution safety model.

---

### **8. Timeout vs Retry vs Circuit Breaker**

| Mechanism       | Role                       |
| --------------- | -------------------------- |
| Timeout         | Stops slow execution       |
| Retry           | Recovers transient failure |
| Circuit breaker | Prevents repeated failure  |
| Fallback        | Provides safe alternative  |

Together they form a **resilience layer**.

---

### **9. Production Design Pattern**

```
LLM Node (15s timeout, 3 retries)
   ↓
Tool Node (5s timeout, 2 retries)
   ↓
Human Review (24h timeout)
   ↓
Finalize
```

---

### **10. Mental Model**

Timeouts transform:

> **Unpredictable execution** → **Deterministic system behavior**

They are essential for **cost control, reliability, scalability, and safety**.

### Demonstration

In [1]:
from typing import TypedDict

class State(TypedDict):
    step: int
    result: str
    done: bool

import time
from langgraph.graph import StateGraph, END

def slow_llm(state):
    time.sleep(6)  # simulate slow model
    return {"result": "draft answer"}

def unreliable_tool(state):
    time.sleep(4)  # simulate slow API
    return {"result": state["result"] + " + tool data"}

def finalize(state):
    return {"done": True}


In [2]:
builder = StateGraph(State)

builder.add_node("llm", slow_llm, timeout=5, retry=2, fallback="finalize")
builder.add_node("tool", unreliable_tool, timeout=3, retry=1, fallback="finalize")
builder.add_node("finalize", finalize)

builder.set_entry_point("llm")
builder.add_edge("llm", "tool")
builder.add_edge("tool", "finalize")
builder.add_edge("finalize", END)

graph = builder.compile()


In [4]:
result = graph.invoke(
    {"step": 0, "done": False},
    config={"timeout": 15}
)
print(result)


{'step': 0, 'result': 'draft answer + tool data', 'done': True}
